### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.fit_manager as fm
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [3]:
from importlib import reload
reload(dl)
reload(sf)
reload(fit)
reload(fm)

<module 'fitter.fit_manager' from '../fitter/fit_manager.py'>

### Specify fit parameters

In [4]:
params = {
    'collection_name' : 'test', 
    
    'model_info' : {
        'fit_type' : 'xpt', # 'ma', 'ma-ratio', 'xpt', 'xpt-ratio', or 'poly'
        'order' : 'n2lo', # 'nlo', 'n2lo', 'n3lo'
        'latt_ct' : 'n3lo', # n2lo : a^2; n3lo : a^4; etc
        'F2' : 'FKFK', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)

        'include_FV' : True, # max 10
        'exclude' : [], # put LECs here
        'use_bijnens_central_value' : True,
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : False, 
        'include_log2' : False,
        'include_sunset' : False,
    }, 

    'fast_susnet' : False,
    'use_prior' : False,
    'bias_correct' : True,
    #'include_su2_isospin_corrrection' : False,
    
    'abbrs' : [u'a06m310L', u'a09m135', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : True,
    
    'save_results' : False,
    'save_pickles' : True,
}

In [5]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()
params['model_info']['name'] = data_loader.get_model_name_from_model_info(params['model_info'])

# Get prior
prior = None
if params['use_prior']:
    prior = data_loader.get_prior(model_info=params['model_info'], collection_name=params['collection_name'])
    
# Make fit
fit_manager = fm.fit_manager(fit_data, phys_point_data, prior=prior, **params)

if params['make_plots']:
    data_loader.save_plots(
        fit_manager.make_plots(show_error_ellipses=params['show_error_ellipses']),
        output_filename=params['collection_name']+'/fits/'+fit_manager.model
    )
else:
    print(fit_manager)
    
if params['save_results']:
    data_loader.save_fit_info(
        fit_manager.fit_info,
        collection_name=params['collection_name'], 
        save_pickles=params['save_pickles']
    )

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.
Making fit...
Done!
Time (s):  1.4168376922607422

Model: xpt_FKFK_n2lo_a4_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1936(38) [1.1957(19)]	(delta_su2 = -0.00418(70))

Parameters:
            L_4    -0.0013 (35)      [   0.0000 (50) ]  
            L_5    0.00027 (86)      [   0.0000 (50) ]  
            A_k       2.5 (6.9)      [        0 (10) ]  
            A_p       0.6 (3.7)      [        0 (10) ]  
            A_a     -11.1 (1.5)      [       0 (100) ]  
           A_aa         81 (23)      [      0 (1000) ]  

Least Square Fit:
  chi2/dof [dof] = 0.84 [18]    Q = 0.66    logGBF = 68.433

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 21/1.4)

Error Budget (relative error):
  disc       7.7e-05
  chiral     6.0e-04
  pp_input   1.2e-03
  stat       2.9e-03


Total time (s):  2.4950907230377197 



In [14]:
fit_manager.plot_parameters(xparam='A_k', show_prior=True);

In [15]:
fit_manager.plot_parameters(xparam='A_k', show_posterior=True);

In [61]:
class Linear(lsqfit.MultiFitterModel):
    def __init__(self, datatag, x, intercept, slope):
        super(Linear, self).__init__(datatag)
        # the independent variable
        self.x = np.array(x)
        # keys used to find the intercept and slope in a parameter dictionary
        self.intercept = intercept
        self.slope = slope

    def fitfcn(self, p):
        try:
            print('Full p', p)
            return p[self.intercept] + p[self.slope] * self.x
        except KeyError:
            print('Marginalized', p)
            print(len(self.x) * [p[self.intercept]])
            # slope parameter marginalized/omitted
            return len(self.x) * [p[self.intercept]]

    def buildprior(self, prior, mopt=None):
        " Extract the model's parameters from prior. "
        newprior = {}
        newprior[self.intercept] = prior[self.intercept]
        if mopt is None:
            # slope parameter marginalized/omitted if mopt is not None
            newprior[self.slope] = prior[self.slope]
            
        print(prior)
        return newprior

    def builddata(self, data):
        " Extract the model's fit data from data. "
        return data[self.datatag]
    
    


In [62]:
data = gv.gvar(dict(
    d1=['1.154(10)', '2.107(16)', '3.042(22)', '3.978(29)'],
    d2=['0.692(10)', '1.196(16)', '1.657(22)', '2.189(29)'],
    d3=['0.107(10)', '0.030(16)', '-0.027(22)', '-0.149(29)'],
    d4=['0.002(10)', '-0.197(16)', '-0.382(22)', '-0.627(29)'],
    ))

models = [Linear('d'+str(j), x=[1, 2, 3, 4], intercept='a', slope='s'+str(j)) for j in range(1, 4)]
prior = {}
prior['a'] = gv.gvar('0(1)')
prior['s1'] = gv.gvar('0(1)')
prior['s2'] = gv.gvar('0(1)')
prior['s3'] = gv.gvar('0(1)')
prior['s4'] = gv.gvar('0(1)')

In [63]:
fitter = lsqfit.MultiFitter(models=models)
fit = fitter.lsqfit(data=data, prior=prior, mopt=True)
print(fit)

{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
Full p {'a': 0.0(1.0),'s1': 0.0(1.0),'s2': 0.0(1.0),'s3': 0.0(1.0)}
Full p {'a': 0.0(1.0),'s1': 0.0(1.0),'s2': 0.0(1.0),'s3': 0.0(1.0)}
Full p {'a': 0.0(1.0),'s1': 0.0(1.0),'s2': 0.0(1.0),'s3': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
{'a': 0.0(1.0), 's1': 0.0(1.0), 's2': 0.0(1.0), 's3': 0.0(1.0), 's4': 0.0(1.0)}
Full p {'a': 0.0(1.0)}
Marginalized {'a': 0.0(1.0)}
[0.0(1.0), 0.0(1.0), 0.0

In [5]:
print(fit_manager.fmt_error_budget())

Partial Errors:
                  FK/Fpi
------------------------
          A_a:   0.00010
         A_aa:   0.00020
        A_aaa:   0.00202
          A_k:   0.00004
          A_p:   0.00000
          L_1:   0.00003
          L_2:   0.00008
          L_3:   0.00032
          L_4:   0.00005
          L_5:   0.00003
          L_6:   0.00000
          L_7:   0.00001
          L_8:   0.00000
     phys:mpi:   0.00013
      phys:mk:   0.00017
phys:lam2_chi:   0.00140
         stat:   0.00407
------------------------
        total:   0.00478



In [18]:
fit_manager.plot_fit(param='mpi');

In [19]:
fit_manager.plot_fit(param='a');

In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


fit_manager.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined